## Harjoitustyö

### Vaihe 1:

#### Hae Digitrafficin REST-rajapintapalvelusta (JSON-formaatissa) aktiiviset junat pääteosoitteella `/live-trains` Helsingin asemalta, joiden hakuparametrit ovat seuraavat:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False


#### Tämän jälkeen lue kyseinen data Pandasin dataframeen ilman muokkaamista, jolloin sarakkeiden otsikot ovat seuraavanlaiset:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version` 

In [1]:
import pandas as pd
import requests
import re
import sqlite3
from sqlalchemy import create_engine

url = 'https://rata.digitraffic.fi/api/v1/live-trains/station/HKI?arrived_trains=10&departed_trains=10&include_nonstopping=false'
req = requests.get(url)
data = req.json()
dataframe = pd.DataFrame(data)
dataframe = dataframe.reindex(sorted(dataframe.columns), axis=1)
dataframe.head(5)

,cancelled,commuterLineID,departureDate,operatorShortCode,operatorUICCode,runningCurrently,timeTableRows,timetableAcceptanceDate,timetableType,trainCategory,trainNumber,trainType,version
0,False,,2020-01-11,vr,10,True,"[{'stationShortCode': 'JNS', 'stationUICCode':...",2019-11-08T10:34:22.000Z,REGULAR,Long-distance,10,IC,266492771137
1,False,,2020-01-11,vr,10,False,"[{'stationShortCode': 'OL', 'stationUICCode': ...",2019-11-08T10:34:22.000Z,REGULAR,Long-distance,26,IC,266492751509
2,False,,2020-01-11,vr,10,True,"[{'stationShortCode': 'HKI', 'stationUICCode':...",2019-11-08T10:34:22.000Z,REGULAR,Long-distance,73,IC,266492773449
3,False,,2020-01-11,vr,10,False,"[{'stationShortCode': 'TKU', 'stationUICCode':...",2019-11-08T10:34:22.000Z,REGULAR,Long-distance,966,S,266492639571
4,False,,2020-01-11,vr,10,False,"[{'stationShortCode': 'HKI', 'stationUICCode':...",2019-11-08T10:34:22.000Z,REGULAR,Long-distance,971,IC,266492655890


#### Poista kaikki muut sarakkeet paitsi `timeTableRows` ja `trainNumber`

In [2]:
dataframe.drop(dataframe.columns.difference(['timeTableRows','trainNumber']), 1, inplace=True)
dataframe

,timeTableRows,trainNumber
0,"[{'stationShortCode': 'JNS', 'stationUICCode':...",10
1,"[{'stationShortCode': 'OL', 'stationUICCode': ...",26
2,"[{'stationShortCode': 'HKI', 'stationUICCode':...",73
3,"[{'stationShortCode': 'TKU', 'stationUICCode':...",966
4,"[{'stationShortCode': 'HKI', 'stationUICCode':...",971
5,"[{'stationShortCode': 'LPV', 'stationUICCode':...",8178
6,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8183
7,"[{'stationShortCode': 'KLH', 'stationUICCode':...",8372
8,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8375
9,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8491


#### Tämän jälkeen pura `timeTableRows` sarakkeen arvot jokaisesta junasta erikseen (sarakkeen arvo jokaiselle junalle sisältää useampia JSON-objekteja) ja lisää dataan mukaan junan tiedot yksilöivä `trainNumber`

In [3]:
data = pd.DataFrame([])
train_numbers = []

for i in range(len(dataframe['timeTableRows'])):
    temp = dataframe['timeTableRows'][i]
    data = data.append(temp, ignore_index = True, sort=True)

for i in range(len(dataframe)):
    for j in range(len(dataframe['timeTableRows'][i])):
        train_numbers.append(dataframe['trainNumber'][i])
        
data['trainNumbers'] = train_numbers

data

,actualTime,cancelled,causes,commercialStop,commercialTrack,countryCode,differenceInMinutes,estimateSource,liveEstimateTime,scheduledTime,stationShortCode,stationUICCode,trainReady,trainStopping,type,trainNumbers
0,2020-01-11T13:13:26.000Z,False,[],True,3,FI,0.0,NaN,NaN,2020-01-11T13:13:00.000Z,JNS,460,"{'source': 'KUPLA', 'accepted': True, 'timesta...",True,DEPARTURE,10
1,2020-01-11T13:14:43.000Z,False,[],NaN,,FI,1.0,NaN,NaN,2020-01-11T13:13:34.000Z,PLT,1070,NaN,False,ARRIVAL,10
2,2020-01-11T13:14:51.000Z,False,[],NaN,,FI,1.0,NaN,NaN,2020-01-11T13:13:34.000Z,PLT,1070,NaN,False,DEPARTURE,10
3,2020-01-11T13:18:00.000Z,False,[],NaN,,FI,4.0,NaN,NaN,2020-01-11T13:14:14.000Z,SUL,1071,NaN,False,ARRIVAL,10
4,2020-01-11T13:15:36.000Z,False,[],NaN,,FI,1.0,NaN,NaN,2020-01-11T13:14:14.000Z,SUL,1071,NaN,False,DEPARTURE,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,NaN,False,[],True,735,FI,NaN,NaN,NaN,2020-01-11T17:34:00.000Z,ILR,1030,NaN,True,ARRIVAL,64502
1672,2020-01-11T17:30:09.000Z,False,[],True,777,FI,0.0,NaN,NaN,2020-01-11T17:30:10.000Z,ILR,1030,NaN,True,DEPARTURE,65012
1673,2020-01-11T17:34:10.000Z,False,[],NaN,,FI,-6.0,NaN,NaN,2020-01-11T17:40:10.000Z,PSL,10,NaN,False,ARRIVAL,65012
1674,2020-01-11T17:35:14.000Z,False,[],NaN,,FI,-5.0,NaN,NaN,2020-01-11T17:40:10.000Z,PSL,10,NaN,False,DEPARTURE,65012


#### Pudota sitten seuraavat sarakkeet datasta: `causes`, `estimateSource`, `liveEstimateTime`, `differenceInMinutes`, `commercialStop` ja `trainReady`

In [4]:
data = data.drop(['causes', 'estimateSource', 'liveEstimateTime', 'differenceInMinutes', 'commercialStop', 'trainReady'], axis=1)
data

,actualTime,cancelled,commercialTrack,countryCode,scheduledTime,stationShortCode,stationUICCode,trainStopping,type,trainNumbers
0,2020-01-11T13:13:26.000Z,False,3,FI,2020-01-11T13:13:00.000Z,JNS,460,True,DEPARTURE,10
1,2020-01-11T13:14:43.000Z,False,,FI,2020-01-11T13:13:34.000Z,PLT,1070,False,ARRIVAL,10
2,2020-01-11T13:14:51.000Z,False,,FI,2020-01-11T13:13:34.000Z,PLT,1070,False,DEPARTURE,10
3,2020-01-11T13:18:00.000Z,False,,FI,2020-01-11T13:14:14.000Z,SUL,1071,False,ARRIVAL,10
4,2020-01-11T13:15:36.000Z,False,,FI,2020-01-11T13:14:14.000Z,SUL,1071,False,DEPARTURE,10
...,...,...,...,...,...,...,...,...,...,...
1671,NaN,False,735,FI,2020-01-11T17:34:00.000Z,ILR,1030,True,ARRIVAL,64502
1672,2020-01-11T17:30:09.000Z,False,777,FI,2020-01-11T17:30:10.000Z,ILR,1030,True,DEPARTURE,65012
1673,2020-01-11T17:34:10.000Z,False,,FI,2020-01-11T17:40:10.000Z,PSL,10,False,ARRIVAL,65012
1674,2020-01-11T17:35:14.000Z,False,,FI,2020-01-11T17:40:10.000Z,PSL,10,False,DEPARTURE,65012


#### Täytä jokaisen rivin sarakkeen `actualTime` puuttuvat arvot `scheduledTime` -sarakkeen arvolla.

In [5]:
data.loc[(pd.isnull(data['actualTime'])), ['actualTime']] = data['scheduledTime']

data

,actualTime,cancelled,commercialTrack,countryCode,scheduledTime,stationShortCode,stationUICCode,trainStopping,type,trainNumbers
0,2020-01-11T13:13:26.000Z,False,3,FI,2020-01-11T13:13:00.000Z,JNS,460,True,DEPARTURE,10
1,2020-01-11T13:14:43.000Z,False,,FI,2020-01-11T13:13:34.000Z,PLT,1070,False,ARRIVAL,10
2,2020-01-11T13:14:51.000Z,False,,FI,2020-01-11T13:13:34.000Z,PLT,1070,False,DEPARTURE,10
3,2020-01-11T13:18:00.000Z,False,,FI,2020-01-11T13:14:14.000Z,SUL,1071,False,ARRIVAL,10
4,2020-01-11T13:15:36.000Z,False,,FI,2020-01-11T13:14:14.000Z,SUL,1071,False,DEPARTURE,10
...,...,...,...,...,...,...,...,...,...,...
1671,2020-01-11T17:34:00.000Z,False,735,FI,2020-01-11T17:34:00.000Z,ILR,1030,True,ARRIVAL,64502
1672,2020-01-11T17:30:09.000Z,False,777,FI,2020-01-11T17:30:10.000Z,ILR,1030,True,DEPARTURE,65012
1673,2020-01-11T17:34:10.000Z,False,,FI,2020-01-11T17:40:10.000Z,PSL,10,False,ARRIVAL,65012
1674,2020-01-11T17:35:14.000Z,False,,FI,2020-01-11T17:40:10.000Z,PSL,10,False,DEPARTURE,65012


### Vaihe 2:

#### Pura tämän kansion sisällä olevan `data.html` -tiedoston `table`-elementin solujen arvot käyttäen regex-syntaksia ja yhdistä ne vaiheessa 1 luotuun dataframeen.

In [6]:
html_text = open('data.html', 'r').read()

re_columns = r'<th[^>]*>+([^<]+)</th>'
re_values = r'<td[^>]*>+\n([^<]+)</td>'

html_columns = re.findall(re_columns, html_text)
html_values = re.findall(re_values, html_text)

parsed_data = [s.translate(str.maketrans({'\n': '', ' ': '', '\t': ''})) for s in html_values]

html_dataframe = pd.DataFrame([],columns=list(html_columns))
temp_list = []
    
for i in range(len(parsed_data)):
    temp_list.append(parsed_data[i])
    if (len(temp_list) == len(html_columns)):
        df = pd.DataFrame([temp_list], columns=list(html_columns))
        html_dataframe = html_dataframe.append(df)
        temp_list = []

data = data.append(html_dataframe, ignore_index=True, sort=True)
data

,actualTime,cancelled,commercialTrack,countryCode,scheduledTime,stationShortCode,stationUICCode,trainNumber,trainNumbers,trainStopping,type
0,2020-01-11T13:13:26.000Z,False,3,FI,2020-01-11T13:13:00.000Z,JNS,460,NaN,10.0,True,DEPARTURE
1,2020-01-11T13:14:43.000Z,False,,FI,2020-01-11T13:13:34.000Z,PLT,1070,NaN,10.0,False,ARRIVAL
2,2020-01-11T13:14:51.000Z,False,,FI,2020-01-11T13:13:34.000Z,PLT,1070,NaN,10.0,False,DEPARTURE
3,2020-01-11T13:18:00.000Z,False,,FI,2020-01-11T13:14:14.000Z,SUL,1071,NaN,10.0,False,ARRIVAL
4,2020-01-11T13:15:36.000Z,False,,FI,2020-01-11T13:14:14.000Z,SUL,1071,NaN,10.0,False,DEPARTURE
...,...,...,...,...,...,...,...,...,...,...,...
1674,2020-01-11T17:35:14.000Z,False,,FI,2020-01-11T17:40:10.000Z,PSL,10,NaN,65012.0,False,DEPARTURE
1675,2020-01-11T17:45:10.000Z,False,5,FI,2020-01-11T17:45:10.000Z,HKI,1,NaN,65012.0,True,ARRIVAL
1676,2019-08-06T04:48:00.000Z,False,,FI,2019-08-06T04:48:00.000Z,JVS,1272,150,NaN,False,DEPARTURE
1677,2019-08-06T06:04:39.000Z,False,,FI,2019-08-06T06:04:39.000Z,R702,98998,150,NaN,False,ARRIVAL


### Vaihe 3:

#### Tee SQLite tietokanta ja lisää sinne vaiheessa 2 luotu dataframe.


In [7]:
try:
    engine = create_engine('sqlite:///trains.db', echo=False)
    data.to_sql('trains_table', con=engine, index=False)
except Exception as e:
    print(e)

#tulostetaan taulukko trains.db -tiedostosta
test_connection = sqlite3.connect('trains.db')
test_dataframe = pd.read_sql('SELECT * FROM trains_table', test_connection)
test_connection.close()
test_dataframe

,actualTime,cancelled,commercialTrack,countryCode,scheduledTime,stationShortCode,stationUICCode,trainNumber,trainNumbers,trainStopping,type
0,2020-01-11T13:13:26.000Z,0,3,FI,2020-01-11T13:13:00.000Z,JNS,460,None,10.0,1,DEPARTURE
1,2020-01-11T13:14:43.000Z,0,,FI,2020-01-11T13:13:34.000Z,PLT,1070,None,10.0,0,ARRIVAL
2,2020-01-11T13:14:51.000Z,0,,FI,2020-01-11T13:13:34.000Z,PLT,1070,None,10.0,0,DEPARTURE
3,2020-01-11T13:18:00.000Z,0,,FI,2020-01-11T13:14:14.000Z,SUL,1071,None,10.0,0,ARRIVAL
4,2020-01-11T13:15:36.000Z,0,,FI,2020-01-11T13:14:14.000Z,SUL,1071,None,10.0,0,DEPARTURE
...,...,...,...,...,...,...,...,...,...,...,...
1674,2020-01-11T17:35:14.000Z,0,,FI,2020-01-11T17:40:10.000Z,PSL,10,None,65012.0,0,DEPARTURE
1675,2020-01-11T17:45:10.000Z,0,5,FI,2020-01-11T17:45:10.000Z,HKI,1,None,65012.0,1,ARRIVAL
1676,2019-08-06T04:48:00.000Z,False,,FI,2019-08-06T04:48:00.000Z,JVS,1272,150,NaN,False,DEPARTURE
1677,2019-08-06T06:04:39.000Z,False,,FI,2019-08-06T06:04:39.000Z,R702,98998,150,NaN,False,ARRIVAL
